# Import Libraries

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'True'

import tequila as tq
from utility import *
import numpy as np
import copy

# Setup Hamiltonian

In [2]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('h2', 2, 'sto-3g', qubit_transf='jw'))

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4)
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(1)X(2)X(3)"), angle=a)
print(U)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))



In [3]:
E = tq.ExpectationValue(H=H, U=U)
vars = {'beta_1': 3.141592624143881, 'beta_0': 3.141592624143881, 'tau_0': 1.1331410014096885, 'gamma_1': 0.0, 'beta_3': 0.0, 'gamma_3': 0.0, 'gamma_2': 0.0, 'gamma_0': 0.0, 'beta_2': 0.0} # values obtained from step 3
print(tq.simulate(E, variables=vars))

-0.9486411121761621


# Setup Measurements

In [4]:
binary_H = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
commuting_parts = binary_H.commuting_groups()

In [5]:
len(commuting_parts)

2

In [6]:
commuting_parts[0].to_qubit_hamiltonian()

-0.5339+0.0673Z(0)+0.0673Z(1)+0.0067Z(2)+0.0067Z(3)+0.1274Z(0)Z(1)+0.0650Z(0)Z(2)+0.1298Z(0)Z(3)+0.1298Z(1)Z(2)+0.0650Z(1)Z(3)+0.1337Z(2)Z(3)

In [7]:
commuting_parts[1].to_qubit_hamiltonian()

-0.0648X(0)X(1)Y(2)Y(3)+0.0648X(0)Y(1)Y(2)X(3)+0.0648Y(0)X(1)X(2)Y(3)-0.0648Y(0)Y(1)X(2)X(3)

## Split Hamiltonians by Measurement Groups

In [8]:
def get_by_part(H, U, part):
    binary_H = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
    commuting_parts = binary_H.commuting_groups()
    result = tq.Objective()
    
    cH = commuting_parts[part]
    
    qwc, Um = cH.get_qubit_wise()
    
    return qwc, Um

In [9]:
H_0, U_0 = get_by_part(H, U, 0)

In [10]:
H_1, U_1 = get_by_part(H, U, 1)

# Simulated Run 

In [11]:
no_samples = 100000

In [12]:
circ_0 = tq.circuit.compiler.compile_exponential_pauli_gate(U + U_0)
tq.draw(circ_0, backend="qiskit")

┌────────────────────┐┌─────────────────────┐┌──────────┐               »
q_0: ┤ Rx(f((beta_0,))_0) ├┤ Rz(f((gamma_0,))_1) ├┤ Ry(-π/2) ├──■────────────»
     ├────────────────────┤├─────────────────────┤├─────────┬┘┌─┴─┐          »
q_1: ┤ Rx(f((beta_1,))_2) ├┤ Rz(f((gamma_1,))_3) ├┤ Rx(π/2) ├─┤ X ├──■───────»
     ├────────────────────┤├─────────────────────┤├─────────┴┐└───┘┌─┴─┐     »
q_2: ┤ Rx(f((beta_2,))_4) ├┤ Rz(f((gamma_2,))_5) ├┤ Ry(-π/2) ├─────┤ X ├──■──»
     ├────────────────────┤├─────────────────────┤├──────────┤     └───┘┌─┴─┐»
q_3: ┤ Rx(f((beta_3,))_6) ├┤ Rz(f((gamma_3,))_7) ├┤ Ry(-π/2) ├──────────┤ X ├»
     └────────────────────┘└─────────────────────┘└──────────┘          └───┘»
c: 4/════════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                     ┌─────────┐ 
«q_0: ──────────────────────────────────────────■─────┤ Ry(π/2) ├─
«                                             ┌─┴─┐   ├─────────┴┐
«q_1: ───────────────────────────────■────────┤ X ├───┤ Rx(-π/2) ├
«                                  ┌─┴─┐   ┌──┴───┴──┐└──────────┘
«q_2: ───────────────────────■─────┤ X ├───┤ Ry(π/2) ├────────────
«     ┌───────────────────┐┌─┴─┐┌──┴───┴──┐└─────────┘            
«q_3: ┤ Rz(f((tau_0,))_8) ├┤ X ├┤ Ry(π/2) ├───────────────────────
«     └───────────────────┘└───┘└─────────┘                       
«c: 4/════════════════════════════════════════════════════════════
«

In [13]:
res_0 = tq.simulate(circ_0, variables=vars, samples = no_samples, backend="qiskit")

In [14]:
res_0

+28837.0000|0011> +71163.0000|1100> 

In [15]:
res_0.state

{3: 28837, 12: 71163}

In [16]:
total_samples = res_0.state[3] + res_0.state[12]
total_samples 

100000

In [17]:
res_0.state[3] = np.sqrt(res_0.state[3]/total_samples)
res_0.state[12] = np.sqrt(res_0.state[12]/total_samples)

In [18]:
res_0

+0.5370|0011> +0.8436|1100> 

In [19]:
E_0 = res_0.compute_expectationvalue(H_0)
E_0

-0.713859458568775

In [20]:
circ_1 = tq.circuit.compiler.compile_exponential_pauli_gate(U + U_1)
tq.draw(circ_1, backend="qiskit")

┌────────────────────┐┌─────────────────────┐┌──────────┐               »
q_0: ┤ Rx(f((beta_0,))_0) ├┤ Rz(f((gamma_0,))_1) ├┤ Ry(-π/2) ├──■────────────»
     ├────────────────────┤├─────────────────────┤├─────────┬┘┌─┴─┐          »
q_1: ┤ Rx(f((beta_1,))_2) ├┤ Rz(f((gamma_1,))_3) ├┤ Rx(π/2) ├─┤ X ├──■───────»
     ├────────────────────┤├─────────────────────┤├─────────┴┐└───┘┌─┴─┐     »
q_2: ┤ Rx(f((beta_2,))_4) ├┤ Rz(f((gamma_2,))_5) ├┤ Ry(-π/2) ├─────┤ X ├──■──»
     ├────────────────────┤├─────────────────────┤├──────────┤     └───┘┌─┴─┐»
q_3: ┤ Rx(f((beta_3,))_6) ├┤ Rz(f((gamma_3,))_7) ├┤ Ry(-π/2) ├──────────┤ X ├»
     └────────────────────┘└─────────────────────┘└──────────┘          └───┘»
c: 4/════════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                     ┌─────────┐      »
«q_0: ──────────────────────────────────────────■─────┤ Ry(π/2) ├───■──»
«                                             ┌─┴─┐   ├─────────┴┐  │  »
«q_1: ───────────────────────────────■────────┤ X ├───┤ Rx(-π/2) ├──┼──»
«                                  ┌─┴─┐   ┌──┴───┴──┐└──────────┘  │  »
«q_2: ───────────────────────■─────┤ X ├───┤ Ry(π/2) ├──────────────┼──»
«     ┌───────────────────┐┌─┴─┐┌──┴───┴──┐└─────────┘            ┌─┴─┐»
«q_3: ┤ Rz(f((tau_0,))_8) ├┤ X ├┤ Ry(π/2) ├───────────────────────┤ X ├»
«     └───────────────────┘└───┘└─────────┘                       └───┘»
«c: 4/═════════════════════════════════════════════════════════════════»
«                                                                      »
«                      ┌──────────┐                      ┌──────────┐»
«q_0: ──────────────■──┤ Rx(-π/2) ├──■────────────────■──┤ Ry(-π/2) ├»
«                   │  └──────────┘  │                │  └──────────┘»
«q_1: ──────────────┼────────────────┼────────────────┼───────■──────»
«                   │                │                │       │      »
«q_2: ──────────────┼────────────────┼────────────────┼───────┼──────»
«     ┌──────────┐┌─┴─┐            ┌─┴─┐┌──────────┐┌─┴─┐   ┌─┴─┐    »
«q_3: ┤ Rz(-π/2) ├┤ X ├────────────┤ X ├┤ Rz(-π/2) ├┤ X ├───┤ X ├────»
«     └──────────┘└───┘            └───┘└──────────┘└───┘   └───┘    »
«c: 4/═══════════════════════════════════════════════════════════════»
«                                                                    »
«                                                                    »
«q_0: ───────────────────────────────────────────────────────────────»
«                      ┌──────────┐                      ┌──────────┐»
«q_1: ──────────────■──┤ Rx(-π/2) ├──■────────────────■──┤ Ry(-π/2) ├»
«                   │  └──────────┘  │                │  └──────────┘»
«q_2: ──────────────┼────────────────┼────────────────┼───────■──────»
«     ┌──────────┐┌─┴─┐            ┌─┴─┐┌──────────┐┌─┴─┐   ┌─┴─┐    »
«q_3: ┤ Rz(-π/2) ├┤ X ├────────────┤ X ├┤ Rz(-π/2) ├┤ X ├───┤ X ├────»
«     └──────────┘└───┘            └───┘└──────────┘└───┘   └───┘    »
«c: 4/═══════════════════════════════════════════════════════════════»
«                                                                    »
«                                                                              »
«q_0: ─────■───────────────────────────────────────────────────────────────────»
«        ┌─┴─┐                                                                 »
«q_1: ───┤ X ├─────────────────────────────────────────────────────────■───────»
«        └───┘         ┌──────────┐                      ┌──────────┐┌─┴─┐     »
«q_2: ──────────────■──┤ Rx(-π/2) ├──■────────────────■──┤ Ry(-π/2) ├┤ X ├──■──»
«     ┌──────────┐┌─┴─┐└──────────┘┌─┴─┐┌──────────┐┌─┴─┐├──────────┤└───┘┌─┴─┐»
«q_3: ┤ Rz(-π/2) ├┤ X ├────────────┤ X ├┤ Rz(-π/2) ├┤ X ├┤ Ry(-π/2) ├─────┤ X ├»
«     └──────────┘└───┘            └───┘└──────────┘└───┘└──────────┘     └───┘»
«c: 4/════════════════════════════════════

In [21]:
res_1 = tq.simulate(circ_1, variables=vars, samples = no_samples, backend="qiskit")

In [22]:
res_1

+4709.0000|1100> +95291.0000|1101> 

In [23]:
res_1.state

{12: 4709, 13: 95291}

In [24]:
total_samples = res_1.state[12] + res_1.state[13]
total_samples 

100000

In [25]:
res_1.state[12] = np.sqrt(res_1.state[12]/total_samples)
res_1.state[13] = np.sqrt(res_1.state[13]/total_samples)

In [26]:
res_1

+0.2170|1100> +0.9762|1101> 

In [27]:
E_1 = res_1.compute_expectationvalue(H_1)
E_1

-0.2347328133167589

In [28]:
E_sim = E_0 + E_1
E_sim

-0.9485922718855339

# On Actual Quantum Device

In [29]:
from qiskit import IBMQ
IBMQ.save_account('22afb9f6e6768d6c7ea6e144cf330191e8bde8a364b592493ea0ce2815cf3f904b0b90e819a4d01164ca59d5480325c3e2a6ff72ef2d2f207619b9c21cb82c4f')

configrc.store_credentials:WARNING:2022-07-15 15:40:03,614: Credentials already present. Set overwrite=True to overwrite.


In [30]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [31]:
provider=IBMQ.get_provider(hub='ibm-q', group='open', project='main')
print(provider.backends())

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>]


## Get the measurements needed

In [32]:
no_samples = 2000

In [43]:
res_0_test = tq.simulate(circ_0, variables=vars, samples = no_samples, backend="qiskit", device='ibmq_quito')

In [44]:
res_0_test

+57.0000|0000> +123.0000|1000> +109.0000|0100> +1020.0000|1100> +48.0000|0010> +26.0000|1010> +23.0000|0110> +33.0000|1110> +27.0000|0001> +18.0000|1001> +16.0000|0101> +45.0000|1101> +400.0000|0011> +18.0000|1011> +26.0000|0111> +11.0000|1111> 

In [36]:
res_1_test = tq.simulate(circ_1, variables=vars, samples = no_samples, backend="qiskit", device='ibmq_quito')

In [37]:
res_1_test

+121.0000|0000> +102.0000|1000> +90.0000|0100> +233.0000|1100> +69.0000|0010> +51.0000|1010> +52.0000|0110> +82.0000|1110> +151.0000|0001> +97.0000|1001> +101.0000|0101> +560.0000|1101> +85.0000|0011> +68.0000|1011> +35.0000|0111> +103.0000|1111> 

## Find Expectation via without post-processing

In [49]:
base_res_0 = copy.deepcopy(res_0_test)

In [50]:
for i in range(16):
    base_res_0.state[i] = np.sqrt(base_res_0.state[i]/no_samples)

In [52]:
base_res_0

+0.1688|0000> +0.2480|1000> +0.2335|0100> +0.7141|1100> +0.1549|0010> +0.1140|1010> +0.1072|0110> +0.1285|1110> +0.1162|0001> +0.0949|1001> +0.0894|0101> +0.1500|1101> +0.4472|0011> +0.0949|1011> +0.1140|0111> +0.0742|1111> 

In [53]:
base_E_0 = base_res_0.compute_expectationvalue(H_0)
base_E_0

-0.6467700690869538

In [54]:
base_res_1 = copy.deepcopy(res_1_test)

In [55]:
for i in range(16):
    base_res_1.state[i] = np.sqrt(base_res_1.state[i]/no_samples)

In [56]:
base_res_1

+0.2460|0000> +0.2258|1000> +0.2121|0100> +0.3413|1100> +0.1857|0010> +0.1597|1010> +0.1612|0110> +0.2025|1110> +0.2748|0001> +0.2202|1001> +0.2247|0101> +0.5292|1101> +0.2062|0011> +0.1844|1011> +0.1323|0111> +0.2269|1111> 

In [57]:
base_E_1 = base_res_1.compute_expectationvalue(H_1)
base_E_1

-0.04457181767954178

In [58]:
base_E = base_E_0 + base_E_1
base_E

-0.6913418867664956

## Find Expectation via with post-processing

In [69]:
post_res_0 = copy.deepcopy(res_0_test)

In [70]:
total_samples_post_0 = post_res_0.state[3] + post_res_0.state[12]
total_samples_post_0

1420

In [71]:
for i in range(16):
    if i == 3:
        post_res_0.state[i] = np.sqrt(post_res_0.state[i]/total_samples_post_0)
    elif i == 12:
        post_res_0.state[i] = np.sqrt(post_res_0.state[i]/total_samples_post_0)
    else: 
        post_res_0.state[i] = 0

post_res_0

+0.0000i|0000> +0.0000i|1000> +0.0000i|0100> +0.8475|1100> +0.0000i|0010> +0.0000i|1010> +0.0000i|0110> +0.0000i|1110> +0.0000i|0001> +0.0000i|1001> +0.0000i|0101> +0.0000i|1101> +0.5307|0011> +0.0000i|1011> +0.0000i|0111> +0.0000i|1111> 

In [73]:
post_E_0 = post_res_0.compute_expectationvalue(H_0)
post_E_0

-0.7154794048100186

In [77]:
post_res_1 = copy.deepcopy(res_1_test)

In [78]:
total_samples_post_1 = post_res_1.state[12] + post_res_1.state[13]
total_samples_post_1

793

In [79]:
for i in range(16):
    if i == 12:
        post_res_1.state[i] = np.sqrt(post_res_1.state[i]/total_samples_post_1)
    elif i == 13:
        post_res_1.state[i] = np.sqrt(post_res_1.state[i]/total_samples_post_1)
    else: 
        post_res_1.state[i] = 0

post_res_1

+0.0000i|0000> +0.0000i|1000> +0.0000i|0100> +0.5421|1100> +0.0000i|0010> +0.0000i|1010> +0.0000i|0110> +0.0000i|1110> +0.0000i|0001> +0.0000i|1001> +0.0000i|0101> +0.8403|1101> +0.0000i|0011> +0.0000i|1011> +0.0000i|0111> +0.0000i|1111> 

In [80]:
post_res_1.state

{0: 0,
 8: 0,
 4: 0,
 12: 0.5420525188256167,
 2: 0,
 10: 0,
 6: 0,
 14: 0,
 1: 0,
 9: 0,
 5: 0,
 13: 0.8403446119508381,
 3: 0,
 11: 0,
 7: 0,
 15: 0}

In [81]:
post_E_1 = post_res_1.compute_expectationvalue(H_1)
post_E_1

-0.10685785786394145

In [82]:
post_E = post_E_0 + post_E_1
post_E

-0.82233726267396